#### El objetivo de esta notebook es extraer las coordenadas de todas las comisarías en la ciudad

In [1]:
# Importamos librerías
import numpy as np
import pandas as pd

In [2]:
# Importamos el dataset y visualizamos
comisarias = pd.read_csv('../Data/comisarias-policia-de-la-ciudad.csv')
comisarias.head(3)

,long,lat,id,nombre,calle,altura,calle2,direccion,telefonos,observaciones,observaciones_2,barrio,comuna,codigo_postal,codigo_postal_argentino
0,-58.373245,-34.601906,1,Comisaria Vecinal 1-D,Lavalle,451,NaN,LAVALLE 451,4322-8033/8221,NaN,NaN,SAN NICOLAS,1,1047.0,C1047AAI
1,-58.424926,-34.628641,37,Comisaria Vecinal 5-B,Muñiz,1250,NaN,MUÑIZ 1250,4922-1229 /4922-3333,NaN,NaN,BOEDO,5,1255.0,C1255ACD
2,-58.439501,-34.608979,39,Comisaria Vecinal 6-A,Diaz Velez,5152,NaN,DIAZ VELEZ AV. 5152,4982-4509/4520,NaN,NaN,CABALLITO,6,1405.0,C1405DCT


In [3]:
# Resumen
comisarias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   long                     49 non-null     float64
 1   lat                      49 non-null     float64
 2   id                       49 non-null     int64  
 3   nombre                   49 non-null     object 
 4   calle                    49 non-null     object 
 5   altura                   49 non-null     int64  
 6   calle2                   1 non-null      object 
 7   direccion                49 non-null     object 
 8   telefonos                47 non-null     object 
 9   observaciones            5 non-null      object 
 10  observaciones_2          0 non-null      float64
 11  barrio                   49 non-null     object 
 12  comuna                   49 non-null     int64  
 13  codigo_postal            46 non-null     float64
 14  codigo_postal_argentino  46 

In [4]:
# Creamos una columna que contenga la latitud y longitud simultáneamente
comisarias['lat-lon'] = list(zip(comisarias.lat, comisarias.long))
comisarias.sample(3)

,long,lat,id,nombre,calle,altura,calle2,direccion,telefonos,observaciones,observaciones_2,barrio,comuna,codigo_postal,codigo_postal_argentino,lat-lon
11,-58.403227,-34.626032,41,Comisaria Vecinal 3-B,Catamarca,1345,NaN,CATAMARCA 1345,4941-3333/4079,NaN,NaN,SAN CRISTOBAL,3,1246.0,C1246AAC,"(-34.626032231837875, -58.4032266269794)"
36,-58.482525,-34.565145,26,Comisaria Vecinal 12-A,Machain,3045,NaN,MACHAIN 3045,4541-8142,NaN,NaN,VILLA URQUIZA,12,1430.0,C1430DYA,"(-34.56514509801703, -58.48252544688682)"
5,-58.380178,-34.594643,10,Comisaria Comunal 1,Suipacha,1156,NaN,SUIPACHA 1156,4393-0076/3333/7058,NaN,NaN,RETIRO,1,1008.0,C1008AAX,"(-34.59464291225725, -58.380178018988076)"


In [5]:
# Nos quedamos solo con los datos de las coordenadas de las comisarías
comisarias_coordenadas = comisarias['lat-lon']
comisarias_coordenadas.sample(3)

21     (-34.64479474193998, -58.41715600739127)
1       (-34.6286410416508, -58.42492623842034)
5     (-34.59464291225725, -58.380178018988076)
Name: lat-lon, dtype: object

In [6]:
# Hay valores duplicados?
comisarias_coordenadas.duplicated().value_counts()

False    49
Name: lat-lon, dtype: int64

In [7]:
# Hay valores nulos?
comisarias_coordenadas.isnull().sum()

0

# Serie lista para pasar: "comisarias_coordenadas"

In [8]:
comisarias_coordenadas

0       (-34.60190617496225, -58.37324471219407)
1        (-34.6286410416508, -58.42492623842034)
2      (-34.60897857012301, -58.439501432968655)
3        (-34.6267799123199, -58.44808721258975)
4       (-34.62038226566912, -58.45322104235826)
5      (-34.59464291225725, -58.380178018988076)
6       (-34.62267005036509, -58.38560777288463)
7      (-34.591136945873416, -58.39235857682059)
8       (-34.62258675877597, -58.39074866828261)
9     (-34.594078522593456, -58.406719292030175)
10        (-34.620278488629, -58.37414712408447)
11      (-34.626032231837875, -58.4032266269794)
12      (-34.58773209769253, -58.41592730316615)
13    (-34.624279357208245, -58.363990210472046)
14      (-34.63416920703777, -58.36058758279343)
15     (-34.63806272113112, -58.374908589749076)
16     (-34.59932055924757, -58.441084301406406)
17       (-34.6019397377213, -58.38889587843118)
18      (-34.64728405029203, -58.37472729162052)
19      (-34.5733085675013, -58.438955799815425)
20     (-34.56014476

### Generación de columnas a partir de los datos de coordenadas geográficas

En esta parte vamos a generar un conjunto de variables basadas en distancias, para lo cual usaremos las columnas de latitud y longitud del dataframe. En este caso vamos a utilizar el modulo distance de la libreria geopy de Python.

In [9]:
import geopy.distance as distance

*Utilizar los siguientes códigos como referencia para "jugar" con las distancias entre los distintos datasets:*

In [10]:
# Vamos a generar la variable en una Series aparte para luego si anexarla al DF

# props = data_props4['lat-lon']
# subte = subtes_raw['lat-lon']

# distancia_s = []
# for a in props:
#     result = [distance.distance(a, j).m for j in subte]
#     distancia_s.append(np.min(result))

In [11]:
# Agregamos la columna al DF
# data_props4['subte'] = distancia_s

In [12]:
comisarias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   long                     49 non-null     float64
 1   lat                      49 non-null     float64
 2   id                       49 non-null     int64  
 3   nombre                   49 non-null     object 
 4   calle                    49 non-null     object 
 5   altura                   49 non-null     int64  
 6   calle2                   1 non-null      object 
 7   direccion                49 non-null     object 
 8   telefonos                47 non-null     object 
 9   observaciones            5 non-null      object 
 10  observaciones_2          0 non-null      float64
 11  barrio                   49 non-null     object 
 12  comuna                   49 non-null     int64  
 13  codigo_postal            46 non-null     float64
 14  codigo_postal_argentino  46 